<a href="https://colab.research.google.com/github/fjadidi2001/Insurance/blob/main/Insurance_ICCKe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Step 0
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Load the dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Load the dataset from Google Drive
file_path = '/content/drive/My Drive/Insurance/telematics_syn.csv'
df = pd.read_csv(file_path)


Mounted at /content/drive


In [5]:
df.head()

,Duration,Insured.age,Insured.sex,Car.age,Marital,Car.use,Credit.score,Region,Annual.miles.drive,Years.noclaims,...,Left.turn.intensity10,Left.turn.intensity11,Left.turn.intensity12,Right.turn.intensity08,Right.turn.intensity09,Right.turn.intensity10,Right.turn.intensity11,Right.turn.intensity12,NB_Claim,AMT_Claim
0,366,45,Male,-1,Married,Commute,609.0,Urban,6213.71,25,...,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1,5100.171753
1,182,44,Female,3,Married,Commute,575.0,Urban,12427.42,20,...,58.0,24.0,11.0,1099.0,615.0,219.0,101.0,40.0,1,883.554840
2,184,48,Female,6,Married,Commute,847.0,Urban,12427.42,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000
3,183,71,Male,6,Married,Private,842.0,Urban,6213.71,43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000
4,183,84,Male,10,Married,Private,856.0,Urban,6213.71,65,...,2.0,0.0,0.0,325.0,111.0,18.0,4.0,2.0,0,0.000000


In [6]:
df.describe()

,Duration,Insured.age,Car.age,Credit.score,Annual.miles.drive,Years.noclaims,Territory,Annual.pct.driven,Total.miles.driven,Pct.drive.mon,...,Left.turn.intensity10,Left.turn.intensity11,Left.turn.intensity12,Right.turn.intensity08,Right.turn.intensity09,Right.turn.intensity10,Right.turn.intensity11,Right.turn.intensity12,NB_Claim,AMT_Claim
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000
mean,314.204060,51.378950,5.639720,800.888870,9124.122908,28.839960,56.531390,0.502294,4833.575303,0.139365,...,551.574010,487.340690,447.758420,843.461830,565.056100,326.654840,246.713120,198.753690,0.04494,137.602253
std,79.746222,15.467075,4.062135,83.382316,3826.144730,16.123717,24.036518,0.299189,4545.943016,0.042807,...,14687.929802,14198.331308,13719.790281,11630.185503,10657.402935,9460.244357,8977.569994,8585.177049,0.21813,1264.320056
min,27.000000,16.000000,-2.000000,422.000000,0.000000,0.000000,11.000000,0.002740,0.095298,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,200.000000,39.000000,2.000000,766.000000,6213.710000,15.000000,35.000000,0.249315,1529.897500,0.120894,...,0.000000,0.000000,0.000000,11.000000,3.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,365.000000,51.000000,5.000000,825.000000,7456.452000,29.000000,62.000000,0.490411,3468.287765,0.137909,...,3.000000,1.000000,0.000000,122.000000,43.000000,7.000000,2.000000,0.000000,0.00000,0.000000
75%,366.000000,63.000000,8.000000,856.000000,12427.420000,41.000000,78.000000,0.753425,6779.876842,0.155203,...,30.000000,9.000000,2.000000,680.000000,321.000000,81.000000,27.000000,9.000000,0.00000,0.000000
max,366.000000,103.000000,20.000000,900.000000,56731.172300,79.000000,91.000000,1.000000,47282.603936,0.998172,...,794380.000000,793926.000000,793170.000000,841210.000000,841207.000000,841200.000000,841176.000000,841144.000000,3.00000,104074.886700


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Duration                100000 non-null  int64  
 1   Insured.age             100000 non-null  int64  
 2   Insured.sex             100000 non-null  object 
 3   Car.age                 100000 non-null  int64  
 4   Marital                 100000 non-null  object 
 5   Car.use                 100000 non-null  object 
 6   Credit.score            100000 non-null  float64
 7   Region                  100000 non-null  object 
 8   Annual.miles.drive      100000 non-null  float64
 9   Years.noclaims          100000 non-null  int64  
 10  Territory               100000 non-null  int64  
 11  Annual.pct.driven       100000 non-null  float64
 12  Total.miles.driven      100000 non-null  float64
 13  Pct.drive.mon           100000 non-null  float64
 14  Pct.drive.tue        

In [ ]:
# Step 2: Data Preprocessing

# Derive the target variable
df['ClaimYN'] = ((df['NB_Claim'] >= 1) & (df['AMT_Claim'] > 1000)).astype(int)

# Drop unnecessary columns
df = df.drop(['NB_Claim', 'AMT_Claim'], axis=1)

# Handle missing values (if any)
df = df.dropna()

# Encode categorical variables
df = pd.get_dummies(df, drop_first=True)

# Separate features and target variable
X = df.drop('ClaimYN', axis=1)
y = df['ClaimYN']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
df.head()

,Duration,Insured.age,Car.age,Credit.score,Annual.miles.drive,Years.noclaims,Territory,Annual.pct.driven,Total.miles.driven,Pct.drive.mon,...,Right.turn.intensity10,Right.turn.intensity11,Right.turn.intensity12,ClaimYN,Insured.sex_Male,Marital_Single,Car.use_Commute,Car.use_Farmer,Car.use_Private,Region_Urban
0,366,45,-1,609.0,6213.71,25,70,0.849315,8864.376247,0.148070,...,0.0,0.0,0.0,1,True,False,True,False,False,True
1,182,44,3,575.0,12427.42,20,26,0.465753,8092.308208,0.147686,...,219.0,101.0,40.0,0,False,False,True,False,False,True
2,184,48,6,847.0,12427.42,14,84,0.520548,3225.832512,0.153735,...,0.0,0.0,0.0,0,False,False,True,False,False,True
3,183,71,6,842.0,6213.71,43,30,0.065753,253.024528,0.106702,...,0.0,0.0,0.0,0,True,False,False,False,True,True
4,183,84,10,856.0,6213.71,65,70,0.441096,4374.379634,0.123807,...,18.0,4.0,2.0,0,True,False,False,False,True,True


In [ ]:
# Access the 'ClaimYN' column from the DataFrame 'df'
df['ClaimYN'].describe()  # Use .describe() for distribution statistics

# Or, if you want the ClaimYN Series:
ClaimYN_series = df['ClaimYN']
ClaimYN_series.describe()

,ClaimYN
count,100000.000000
mean,0.026980
std,0.162026
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [1]:
# Enhanced Dual-Path Machine Learning Framework for Driver Risk Assessment
# Complete Step-by-Step Implementation with Innovations

# =============================================================================
# STEP 1: IMPORT REQUIRED LIBRARIES
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import defaultdict
import time
import joblib
from datetime import datetime

# Machine Learning
from sklearn.model_selection import (
    train_test_split, RandomizedSearchCV, cross_val_score,
    StratifiedKFold, learning_curve
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import (
    mutual_info_classif, SelectKBest, RFE, VarianceThreshold
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, matthews_corrcoef, confusion_matrix,
    roc_curve, precision_recall_curve, average_precision_score,
    classification_report, log_loss
)
from sklearn.inspection import permutation_importance, partial_dependence
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

# Class Imbalance Handling
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    Input, Dense, Dropout, BatchNormalization, LSTM,
    GRU, Attention, MultiHeadAttention, LayerNormalization
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2

# Interpretability
try:
    import shap
    import lime
    import lime.lime_tabular
    INTERPRETABILITY_AVAILABLE = True
except ImportError:
    INTERPRETABILITY_AVAILABLE = False
    print("SHAP/LIME not available. Install for interpretability features.")

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
np.random.seed(42)
tf.random.set_seed(42)

# =============================================================================
# STEP 2: ENHANCED xLSTM IMPLEMENTATION
# =============================================================================

class xLSTMCell(tf.keras.layers.Layer):
    """
    Enhanced xLSTM Cell with exponential gating and matrix memory
    Innovation: Implements advanced memory mechanisms for temporal patterns
    """
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.state_size = [units, units, units]  # h, c, m (matrix memory)

    def build(self, input_shape):
        # Input gates with exponential gating
        self.W_i = self.add_weight(shape=(input_shape[-1], self.units),
                                  name='W_i', initializer='glorot_uniform')
        self.U_i = self.add_weight(shape=(self.units, self.units),
                                  name='U_i', initializer='orthogonal')
        self.b_i = self.add_weight(shape=(self.units,), name='b_i',
                                  initializer='zeros')

        # Forget gates with enhanced memory control
        self.W_f = self.add_weight(shape=(input_shape[-1], self.units),
                                  name='W_f', initializer='glorot_uniform')
        self.U_f = self.add_weight(shape=(self.units, self.units),
                                  name='U_f', initializer='orthogonal')
        self.b_f = self.add_weight(shape=(self.units,), name='b_f',
                                  initializer='ones')

        # Candidate values
        self.W_c = self.add_weight(shape=(input_shape[-1], self.units),
                                  name='W_c', initializer='glorot_uniform')
        self.U_c = self.add_weight(shape=(self.units, self.units),
                                  name='U_c', initializer='orthogonal')
        self.b_c = self.add_weight(shape=(self.units,), name='b_c',
                                  initializer='zeros')

        # Output gates
        self.W_o = self.add_weight(shape=(input_shape[-1], self.units),
                                  name='W_o', initializer='glorot_uniform')
        self.U_o = self.add_weight(shape=(self.units, self.units),
                                  name='U_o', initializer='orthogonal')
        self.b_o = self.add_weight(shape=(self.units,), name='b_o',
                                  initializer='zeros')

        # Matrix memory weights (Innovation)
        self.W_m = self.add_weight(shape=(input_shape[-1], self.units),
                                  name='W_m', initializer='glorot_uniform')
        self.U_m = self.add_weight(shape=(self.units, self.units),
                                  name='U_m', initializer='orthogonal')

        super().build(input_shape)

    def call(self, inputs, states):
        h_prev, c_prev, m_prev = states

        # Exponential input gate (Innovation)
        i = tf.nn.sigmoid(tf.matmul(inputs, self.W_i) +
                         tf.matmul(h_prev, self.U_i) + self.b_i)
        i = tf.exp(i) / (tf.exp(i) + 1)  # Exponential gating

        # Enhanced forget gate with matrix memory influence
        f = tf.nn.sigmoid(tf.matmul(inputs, self.W_f) +
                         tf.matmul(h_prev, self.U_f) +
                         tf.reduce_mean(m_prev, axis=-1, keepdims=True) + self.b_f)

        # Candidate values
        c_candidate = tf.nn.tanh(tf.matmul(inputs, self.W_c) +
                               tf.matmul(h_prev, self.U_c) + self.b_c)

        # Matrix memory update (Innovation)
        m_candidate = tf.nn.tanh(tf.matmul(inputs, self.W_m) +
                               tf.matmul(h_prev, self.U_m))
        m_new = f * m_prev + i * tf.expand_dims(m_candidate, axis=-1)

        # Cell state update with matrix memory influence
        c_new = f * c_prev + i * c_candidate * tf.reduce_mean(m_new, axis=-1)

        # Output gate
        o = tf.nn.sigmoid(tf.matmul(inputs, self.W_o) +
                         tf.matmul(h_prev, self.U_o) + self.b_o)

        # Hidden state with matrix memory
        h_new = o * tf.nn.tanh(c_new + tf.reduce_mean(m_new, axis=-1))

        return h_new, [h_new, c_new, m_new]

class xLSTMLayer(tf.keras.layers.Layer):
    """xLSTM Layer wrapper for sequential processing"""
    def __init__(self, units, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.return_sequences = return_sequences
        self.cell = xLSTMCell(units)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        seq_len = tf.shape(inputs)[1]

        # Initialize states
        h = tf.zeros((batch_size, self.units))
        c = tf.zeros((batch_size, self.units))
        m = tf.zeros((batch_size, self.units, self.units))
        states = [h, c, m]

        outputs = []
        for t in range(seq_len):
            output, states = self.cell(inputs[:, t, :], states)
            outputs.append(output)

        if self.return_sequences:
            return tf.stack(outputs, axis=1)
        else:
            return outputs[-1]

# =============================================================================
# STEP 3: COMPREHENSIVE DATA PREPROCESSING CLASS
# =============================================================================

class EnhancedDataPreprocessor:
    """
    Comprehensive data preprocessing with multiple scaling strategies
    and advanced feature engineering
    """

    def __init__(self, scaling_method='standard', handle_outliers=True):
        self.scaling_method = scaling_method
        self.handle_outliers = handle_outliers
        self.scalers = {}
        self.feature_names = None
        self.categorical_features = []
        self.numerical_features = []

    def detect_feature_types(self, df):
        """Automatically detect categorical and numerical features"""
        categorical = df.select_dtypes(include=['object', 'category']).columns.tolist()
        numerical = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
        return categorical, numerical

    def handle_missing_values(self, df):
        """Advanced missing value handling"""
        # Handle negative Car.age values as missing
        if 'Car.age' in df.columns:
            df.loc[df['Car.age'] < 0, 'Car.age'] = np.nan
            df['Car.age'].fillna(df['Car.age'].median(), inplace=True)

        # Fill other missing values
        for col in df.columns:
            if df[col].isnull().sum() > 0:
                if df[col].dtype in ['object', 'category']:
                    df[col].fillna(df[col].mode().iloc[0], inplace=True)
                else:
                    df[col].fillna(df[col].median(), inplace=True)

        return df

    def remove_outliers(self, df, method='iqr', threshold=3):
        """Remove outliers using IQR or Z-score method"""
        if not self.handle_outliers:
            return df

        numerical_cols = df.select_dtypes(include=[np.number]).columns

        if method == 'iqr':
            for col in numerical_cols:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

        elif method == 'zscore':
            for col in numerical_cols:
                z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
                df = df[z_scores < threshold]

        return df

    def log_transform_skewed_features(self, df, skewness_threshold=0.75):
        """Apply log transformation to highly skewed features"""
        numerical_cols = df.select_dtypes(include=[np.number]).columns

        for col in numerical_cols:
            if df[col].min() >= 0:  # Can only log-transform non-negative values
                skewness = df[col].skew()
                if abs(skewness) > skewness_threshold:
                    # Add small constant to handle zeros
                    df[f'{col}_log'] = np.log1p(df[col])

        return df

    def create_interaction_features(self, df):
        """Create interaction features for behavioral metrics"""
        # Acceleration-Brake intensity combinations
        accel_cols = [col for col in df.columns if 'Accel' in col]
        brake_cols = [col for col in df.columns if 'Brake' in col]

        for accel_col in accel_cols:
            for brake_col in brake_cols:
                if accel_col.split('.')[-1] == brake_col.split('.')[-1]:  # Same mile threshold
                    df[f'{accel_col}_x_{brake_col}'] = df[accel_col] * df[brake_col]

        # Turn intensity ratios
        left_turn_cols = [col for col in df.columns if 'Left.turn.intensity' in col]
        right_turn_cols = [col for col in df.columns if 'Right.turn.intensity' in col]

        for left_col in left_turn_cols:
            for right_col in right_turn_cols:
                if left_col.split('intensity')[-1] == right_col.split('intensity')[-1]:
                    # Avoid division by zero
                    df[f'{left_col}_ratio_{right_col}'] = (
                        df[left_col] / (df[right_col] + 1e-8)
                    )

        return df

    def fit_transform(self, X, y=None):
        """Fit preprocessor and transform data"""
        df = X.copy()

        # Store original feature names
        self.feature_names = df.columns.tolist()

        # Detect feature types
        self.categorical_features, self.numerical_features = self.detect_feature_types(df)

        # Handle missing values
        df = self.handle_missing_values(df)

        # Remove outliers
        if y is not None:
            original_indices = df.index
            df = self.remove_outliers(df)
            # Filter y to match filtered df
            y = y.loc[df.index] if hasattr(y, 'loc') else y[df.index]

        # Log transform skewed features
        df = self.log_transform_skewed_features(df)

        # Create interaction features
        df = self.create_interaction_features(df)

        # One-hot encode categorical features
        if self.categorical_features:
            df = pd.get_dummies(df, columns=self.categorical_features, drop_first=True)

        # Update numerical features after transformations
        self.numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()

        # Apply scaling
        if self.scaling_method == 'standard':
            scaler = StandardScaler()
        elif self.scaling_method == 'robust':
            scaler = RobustScaler()
        elif self.scaling_method == 'minmax':
            scaler = MinMaxScaler()
        else:
            scaler = StandardScaler()

        df[self.numerical_features] = scaler.fit_transform(df[self.numerical_features])
        self.scalers['numerical'] = scaler

        return df, y

    def transform(self, X):
        """Transform new data using fitted preprocessor"""
        df = X.copy()

        # Handle missing values
        df = self.handle_missing_values(df)

        # Log transform skewed features
        df = self.log_transform_skewed_features(df)

        # Create interaction features
        df = self.create_interaction_features(df)

        # One-hot encode categorical features
        if self.categorical_features:
            df = pd.get_dummies(df, columns=self.categorical_features, drop_first=True)

        # Ensure all columns are present
        for col in self.numerical_features:
            if col not in df.columns:
                df[col] = 0

        # Apply scaling
        if 'numerical' in self.scalers:
            df[self.numerical_features] = self.scalers['numerical'].transform(
                df[self.numerical_features]
            )

        return df

# =============================================================================
# STEP 4: ADVANCED CLASS IMBALANCE HANDLER
# =============================================================================

class AdvancedImbalanceHandler:
    """
    Advanced class imbalance handling with multiple techniques
    """

    def __init__(self, method='smote_tomek', random_state=42):
        self.method = method
        self.random_state = random_state
        self.sampler = None

    def get_sampler(self):
        """Get appropriate sampler based on method"""
        if self.method == 'smote':
            return SMOTE(random_state=self.random_state)
        elif self.method == 'smote_tomek':
            return SMOTETomek(random_state=self.random_state)
        elif self.method == 'adasyn':
            return ADASYN(random_state=self.random_state)
        elif self.method == 'borderline_smote':
            return BorderlineSMOTE(random_state=self.random_state)
        else:
            return SMOTE(random_state=self.random_state)

    def fit_resample(self, X, y):
        """Apply resampling to training data"""
        self.sampler = self.get_sampler()
        X_resampled, y_resampled = self.sampler.fit_resample(X, y)

        print(f"Original class distribution: {np.bincount(y)}")
        print(f"Resampled class distribution: {np.bincount(y_resampled)}")

        return X_resampled, y_resampled

# =============================================================================
# STEP 5: ENHANCED FEATURE SELECTOR
# =============================================================================

class EnhancedFeatureSelector:
    """
    Multi-method feature selection with ensemble approach
    """

    def __init__(self, n_features=20, methods=['mutual_info', 'rfe', 'variance']):
        self.n_features = n_features
        self.methods = methods
        self.selected_features = None
        self.feature_scores = {}

    def mutual_info_selection(self, X, y):
        """Feature selection using mutual information"""
        mi_scores = mutual_info_classif(X, y, random_state=42)
        feature_scores = dict(zip(X.columns, mi_scores))
        top_features = sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)
        return [feat[0] for feat in top_features[:self.n_features]], feature_scores

    def rfe_selection(self, X, y):
        """Recursive Feature Elimination"""
        estimator = GradientBoostingClassifier(n_estimators=50, random_state=42)
        selector = RFE(estimator, n_features_to_select=self.n_features)
        selector.fit(X, y)

        feature_scores = dict(zip(X.columns, selector.ranking_))
        selected_features = X.columns[selector.support_].tolist()
        return selected_features, feature_scores

    def variance_selection(self, X, threshold=0.01):
        """Remove low-variance features"""
        selector = VarianceThreshold(threshold=threshold)
        selector.fit(X)
        selected_features = X.columns[selector.get_support()].tolist()
        feature_scores = dict(zip(X.columns, selector.variances_))
        return selected_features, feature_scores

    def permutation_importance_selection(self, X, y):
        """Feature selection using permutation importance"""
        estimator = GradientBoostingClassifier(n_estimators=50, random_state=42)
        estimator.fit(X, y)

        perm_importance = permutation_importance(estimator, X, y, n_repeats=5, random_state=42)
        feature_scores = dict(zip(X.columns, perm_importance.importances_mean))
        top_features = sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)
        return [feat[0] for feat in top_features[:self.n_features]], feature_scores

    def ensemble_selection(self, X, y):
        """Ensemble feature selection combining multiple methods"""
        all_selected_features = []
        method_scores = {}

        if 'mutual_info' in self.methods:
            features, scores = self.mutual_info_selection(X, y)
            all_selected_features.extend(features)
            method_scores['mutual_info'] = scores

        if 'rfe' in self.methods:
            features, scores = self.rfe_selection(X, y)
            all_selected_features.extend(features)
            method_scores['rfe'] = scores

        if 'variance' in self.methods:
            features, scores = self.variance_selection(X)
            all_selected_features.extend(features)
            method_scores['variance'] = scores

        if 'permutation' in self.methods:
            features, scores = self.permutation_importance_selection(X, y)
            all_selected_features.extend(features)
            method_scores['permutation'] = scores

        # Count feature frequency across methods
        feature_counts = defaultdict(int)
        for feature in all_selected_features:
            feature_counts[feature] += 1

        # Select features that appear in multiple methods
        ensemble_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)
        self.selected_features = [feat[0] for feat in ensemble_features[:self.n_features]]
        self.feature_scores = method_scores

        return self.selected_features, method_scores

# =============================================================================
# STEP 6: ENHANCED GRADIENT BOOSTING MODEL
# =============================================================================

class EnhancedGradientBoosting:
    """Enhanced Gradient Boosting with advanced hyperparameter optimization"""

    def __init__(self):
        self.model = None
        self.best_params = None
        self.cv_results = None

    def get_param_grid(self):
        """Enhanced parameter grid for optimization"""
        return {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7, 9],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'subsample': [0.8, 0.9, 1.0],
            'max_features': ['sqrt', 'log2', None]
        }

    def optimize_hyperparameters(self, X, y, cv_folds=5, n_iter=50):
        """Optimize hyperparameters using RandomizedSearchCV"""
        gb_classifier = GradientBoostingClassifier(random_state=42)

        search = RandomizedSearchCV(
            estimator=gb_classifier,
            param_distributions=self.get_param_grid(),
            n_iter=n_iter,
            cv=StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42),
            scoring='roc_auc',
            n_jobs=-1,
            random_state=42,
            verbose=1
        )

        search.fit(X, y)
        self.best_params = search.best_params_
        self.cv_results = search.cv_results_
        self.model = search.best_estimator_

        print("Best Gradient Boosting Parameters:")
        for param, value in self.best_params.items():
            print(f"  {param}: {value}")
        print(f"Best CV Score: {search.best_score_:.4f}")

        return self.model

# =============================================================================
# STEP 7: ENHANCED NEURAL NETWORK WITH xLSTM
# =============================================================================

class EnhancedNeuralNetwork:
    """Enhanced Neural Network with xLSTM integration"""

    def __init__(self, include_xlstm=True):
        self.model = None
        self.history = None
        self.include_xlstm = include_xlstm

    def create_mlp_model(self, input_dim, hidden_layers=[128, 64, 32]):
        """Create Multi-Layer Perceptron model"""
        inputs = Input(shape=(input_dim,))
        x = inputs

        for i, units in enumerate(hidden_layers):
            x = Dense(units, activation='relu',
                     kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)
            x = BatchNormalization()(x)
            x = Dropout(0.3)(x)

        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

    def create_xlstm_enhanced_model(self, input_dim, sequence_features=None):
        """Create xLSTM-enhanced model for sequential features"""
        if sequence_features is None:
            sequence_features = ['Daily.Driving.Percentage', 'Night.Driving.Percentage',
                               'Avg.Speed.Per.Trip']

        # Main input
        main_input = Input(shape=(input_dim,), name='main_input')

        # Sequential features processing with xLSTM
        if self.include_xlstm:
            # Reshape sequential features for xLSTM processing
            seq_input = Input(shape=(len(sequence_features), 1), name='seq_input')
            xlstm_output = xLSTMLayer(64, return_sequences=False)(seq_input)
            xlstm_dense = Dense(32, activation='relu')(xlstm_output)

            # Combine main features with xLSTM output
            combined = keras.layers.concatenate([main_input, xlstm_dense])
        else:
            combined = main_input

        # MLP layers
        x = Dense(128, activation='relu',
                 kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(combined)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        x = Dense(64, activation='relu',
                 kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        x = Dense(32, activation='relu')(x)
        x = Dropout(0.2)(x)

        outputs = Dense(1, activation='sigmoid')(x)

        if self.include_xlstm:
            model = Model(inputs=[main_input, seq_input], outputs=outputs)
        else:
            model = Model(inputs=main_input, outputs=outputs)

        return model

    def compile_and_train(self, model, X_train, y_train, X_val, y_val,
                         epochs=100, batch_size=32):
        """Compile and train the neural network"""
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)
        ]

        if isinstance(X_train, list):  # xLSTM model with multiple inputs
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                verbose=1
            )
        else:  # Standard MLP model
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                verbose=1
            )

        self.model = model
        self.history = history
        return model, history

# =============================================================================
# STEP 8: WEIGHTED ENSEMBLE INTEGRATOR
# =============================================================================

class WeightedEnsemble:
    """Advanced weighted ensemble with dynamic weight calculation"""

    def __init__(self, models, weight_method='accuracy'):
        self.models = models
        self.weight_method = weight_method
        self.weights = None

    def calculate_weights(self, X_val, y_val):
        """Calculate dynamic weights based on validation performance"""
        accuracies = {}

        for name, model in self.models.items():
            if hasattr(model, 'predict_proba'):
                y_pred = model.predict_proba(X_val)[:, 1]
            else:
                y_pred = model.predict(X_val)

            if self.weight_method == 'accuracy':
                y_pred_binary = (y_pred > 0.5).astype(int)
                accuracies[name] = accuracy_score(y_val, y_pred_binary)
            elif self.weight_method == 'auc':
                accuracies[name] = roc_auc_score(y_val, y_pred)
            elif self.weight_method == 'f1':
                y_pred_binary = (y_pred > 0.5).astype(int)
                accuracies[name] = f1_score(y_val, y_pred_binary)

        # Normalize weights
        total_accuracy = sum(accuracies.values())
        self.weights = {name: acc/total_accuracy for name, acc in accuracies.items()}

        print("Ensemble Weights:")
        for name, weight in self.weights.items():
            print(f"  {name}: {weight:.4f}")

        return self.weights

    def predict_proba(self, X):
        """Generate ensemble predictions"""
        predictions = {}

        for name, model in self.models.items():
            if hasattr(model, 'predict_proba'):
                predictions[name] = model.predict_proba(X)[:, 1]
            else:
                predictions[name] = model.predict(X)

        # Weighted average
        ensemble_pred = np.zeros(len(X))
        for name, pred in predictions.items():
            ensemble_pred += self.weights[name] * pred

        return ensemble_pred

    def predict(self, X, threshold=0.5):
        """Generate binary predictions"""
        proba = self.predict_proba(X)
        return (proba > threshold).astype(int)

# =============================================================================
# STEP 9: THRESHOLD OPTIMIZER
# =============================================================================

class ThresholdOptimizer:
    """Optimize classification threshold for specific metrics"""

    def __init__(self, metric='f1'):
        self.metric = metric
        self.optimal_threshold = 0.5

    def optimize(self, y_true, y_pred_proba):
        """Find optimal threshold"""
        thresholds = np.linspace(0.01, 0.99, 100)
        best_score = 0
        best_threshold = 0.5

        for threshold in thresholds:
            y_pred = (y_pred_proba >= threshold).astype(int)

            if self.metric == 'f1':
                score = f1_score(y_true, y_pred, zero_division=0)
            elif self.metric == 'precision':
                score = precision_score(y_true, y_pred, zero_division=0)
            elif self.metric == 'recall':
                score = recall_score(y_true, y_pred, zero_division=0)
            elif self.metric == 'balanced':
                precision = precision_score(y_true, y_pred, zero_division=0)
                recall = recall_score(y_true, y_pred, zero_division=0)
                score = 2 * (precision * recall) / (precision + recall + 1e-8)

            if score > best_score:
                best_score = score
                best_threshold = threshold

        self.optimal_threshold = best_threshold
        print(f"Optimal threshold for {self.metric}: {best_threshold:.4f}")
        print(f"Best {self.metric} score: {best_score:.4f}")

        return best_threshold

# =============================================================================
# STEP 10: COMPREHENSIVE MODEL EVALUATOR
# =============================================================================

class ComprehensiveEvaluator:
    """Comprehensive model evaluation with advanced metrics and visualizations"""

    def __init__(self):
        self.results = {}

    def calculate_all_metrics(self, y_true, y_pred, y_pred_proba, model_name):
        """Calculate comprehensive performance metrics"""
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1': f1_score(y_true, y_pred, zero_division=0),
            'auc_roc': roc_auc_score(y_true, y_pred_proba),
            'mcc': matthews_corrcoef(y_true, y_pred),
            'log_loss': log_loss(y_true, y_pred_proba),
            'avg_precision': average_precision_score(y_true, y_pred_proba)
        }

        self.results[model_name] = metrics
        return metrics

    def plot_comprehensive_evaluation(self, y_true, y_pred_proba_dict):
        """Create comprehensive evaluation plots"""
        n_models = len(y_pred_proba_dict)
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))

        # ROC Curves
        ax = axes[0, 0]
        for model_name, y_pred_proba in y_pred_proba_dict.items():
            fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
            auc_score = roc_auc_score(y_true, y_pred_proba)
            ax.plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f})')

        ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC Curves')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Precision-Recall Curves
        ax = axes[0, 1]
        for model_name, y_pred_proba in y_pred_proba_dict.items():
            precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
            avg_precision = average_precision_score(y_true, y_pred_proba)
            ax.plot(recall, precision, label=f'{model_name} (AP = {avg_precision:.3f})')

        ax.set_xlabel('Recall')
        ax.set_ylabel('Precision')
        ax.set_title('Precision-Recall Curves')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Model Comparison Bar Chart
        ax = axes[0, 2]
        metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc_roc']
        x = np.arange(len(metrics))
        width = 0.8 / len(y_pred_proba_dict)

        for i, (model_name, _) in enumerate(y_pred_proba_dict.items()):
            if model_name in self.results:
                values = [self.results[model_name][metric] for metric in metrics]
                ax.bar(x + i * width, values, width, label=model_name, alpha=0.8)

        ax.set_xlabel('Metrics')
        ax.set_ylabel('Score')
        ax.set_title('Model Comparison')
        ax.set_xticks(x + width/2)
        ax.set_xticklabels(metrics, rotation=45)
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Confusion Matrices
        if len(y_pred_proba_dict) <= 3:
            for i, (model_name, y_pred_proba) in enumerate(y_pred_proba_dict.items()):
                ax = axes[1, i]
                y_pred = (y_pred_proba > 0.5).astype(int)
                cm = confusion_matrix(y_true, y_pred)
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
                ax.set_title(f'{model_name} Confusion Matrix')
                ax.set_xlabel('Predicted')
                ax.set_ylabel('Actual')

        plt.tight_layout()
        plt.show()

    def print_results_table(self):
        """Print formatted results table"""
        if not self.results:
            print("No results to display")
            return

        # Create DataFrame for better formatting
        df_results = pd.DataFrame(self.results).T
        df_results = df_results.round(4)

        print("\n" + "="*80)
        print("COMPREHENSIVE MODEL EVALUATION RESULTS")
        print("="*80)
        print(df_results.to_string())
        print("="*80)

# =============================================================================
# STEP 11: MODEL INTERPRETABILITY SUITE
# =============================================================================

class InterpretabilitySuite:
    """Comprehensive model interpretability using SHAP and LIME"""

    def __init__(self, models, feature_names):
        self.models = models
        self.feature_names = feature_names
        self.explainers = {}

    def setup_shap_explainers(self, X_background, sample_size=100):
        """Setup SHAP explainers for all models"""
        if not INTERPRETABILITY_AVAILABLE:
            print("SHAP not available. Install shap package for interpretability.")
            return

        # Sample background data
        if len(X_background) > sample_size:
            background_sample = shap.sample(X_background, sample_size)
        else:
            background_sample = X_background

        for model_name, model in self.models.items():
            try:
                if hasattr(model, 'predict_proba'):
                    self.explainers[model_name] = shap.KernelExplainer(
                        lambda x: model.predict_proba(x)[:, 1],
                        background_sample
                    )
                else:
                    self.explainers[model_name] = shap.KernelExplainer(
                        model.predict, background_sample
                    )
                print(f"SHAP explainer ready for {model_name}")
            except Exception as e:
                print(f"Failed to create SHAP explainer for {model_name}: {e}")

    def explain_predictions(self, X_explain, model_name, max_display=10):
        """Generate SHAP explanations for predictions"""
        if not INTERPRETABILITY_AVAILABLE or model_name not in self.explainers:
            print(f"SHAP explainer not available for {model_name}")
            return None

        try:
            shap_values = self.explainers[model_name].shap_values(X_explain)

            # Summary plot
            plt.figure(figsize=(10, 6))
            shap.summary_plot(shap_values, X_explain,
                            feature_names=self.feature_names[:X_explain.shape[1]],
                            max_display=max_display, show=False)
            plt.title(f'SHAP Summary Plot - {model_name}')
            plt.show()

            return shap_values
        except Exception as e:
            print(f"Failed to generate SHAP explanations for {model_name}: {e}")
            return None

    def setup_lime_explainer(self, X_train):
        """Setup LIME explainer"""
        if not INTERPRETABILITY_AVAILABLE:
            print("LIME not available. Install lime package for interpretability.")
            return None

        try:
            explainer = lime.lime_tabular.LimeTabularExplainer(
                X_train,
                feature_names=self.feature_names[:X_train.shape[1]],
                class_names=['No Claim', 'Claim'],
                mode='classification'
            )
            return explainer
        except Exception as e:
            print(f"Failed to create LIME explainer: {e}")
            return None

    def explain_instance_lime(self, lime_explainer, instance, model, model_name):
        """Explain single instance using LIME"""
        if lime_explainer is None:
            print("LIME explainer not available")
            return None

        try:
            if hasattr(model, 'predict_proba'):
                explanation = lime_explainer.explain_instance(
                    instance, model.predict_proba, num_features=10
                )
            else:
                explanation = lime_explainer.explain_instance(
                    instance, lambda x: np.column_stack([1-model.predict(x), model.predict(x)]),
                    num_features=10
                )

            explanation.show_in_notebook(show_table=True)
            return explanation
        except Exception as e:
            print(f"Failed to generate LIME explanation: {e}")
            return None


SHAP/LIME not available. Install for interpretability features.


In [3]:
!pip install SHAP lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=49448c7cd83b907e7f4c3feb09ed48146e05fffe5ff8b02075ec0259d19702f6
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime
